In [1]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, GenerationConfig

In [ ]:
# This recipe requires a HuggingFace token to access the mistral models from the hub. You will need to create a HuggingFace login
# and generate a token for use at https://huggingface.co/settings/tokens. You will also need to visit the model card at 
# https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.1 and accept the terms to use the model. Once you have generated the token,
# save it in an environment variable named HUGGINGFACE_TOKEN and read it in the recipe as shown in the snippet below.
from huggingface_hub import login
hf_token = os.environ.get('HUGGINGFACE_TOKEN')
login(token=hf_token)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

quantization_config = BitsAndBytesConfig(load_in_4bit=True)
model = AutoModelForCausalLM.from_pretrained(
            "mistralai/Mistral-7B-v0.1", 
            device_map="auto", 
            quantization_config=quantization_config,
        )

tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1", padding_side="left")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
generation_config = GenerationConfig(
    num_beams=4,
    early_stopping=True,
    eos_token_id=model.config.eos_token_id,
    pad_token_id=model.config.eos_token_id,
    max_new_tokens=900,
)

In [5]:
seed_sentence = "Step by step way on how to make an apple pie:"

In [6]:
model_inputs = tokenizer([seed_sentence], return_tensors="pt").to(device)
generated_ids = model.generate(**model_inputs, generation_config=generation_config)

In [7]:
generated_tokens = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_tokens)

Step by step way on how to make an apple pie:

1. Preheat the oven to 200°C/400°F/gas mark 6.
2. Roll out the pastry on a lightly floured surface to the thickness of a £1 coin.
3. Line a 23cm/9in fluted tart tin with the pastry, leaving the excess pastry hanging over the edge.
4. Prick the base of the pastry with a fork.
5. Line the pastry with baking parchment and fill with baking beans.
6. Bake for 15 minutes.
7. Remove the baking beans and baking parchment.
8. Bake for a further 10 minutes, or until the pastry is golden-brown.
9. Remove from the oven and leave to cool.
10. Reduce the oven temperature to 180°C/350°F/gas mark 4.
11. Peel, core and slice the apples.
12. Place the apples in a large bowl.
13. Add the sugar, lemon juice and cinnamon.
14. Mix well.
15. Spoon the apple mixture into the pastry case.
16. Roll out the remaining pastry on a lightly floured surface to the thickness of a £1 coin.
17. Cut the pastry into 2.5cm/1in strips.
18. Weave the pastry strips over the apple